# A comparison between the SMW-iteration and the GMRES method

### 18 July 2023
### Dimitrios Mitsotakis

In [1]:
import numpy as np
import scipy as sp
import scipy.sparse as sps
import scipy.sparse.linalg as spsl
import numpy.linalg as npl
import scipy.linalg as spl
from numpy.fft import fft, ifft

A trivial implementation of the SMW-iteration:

In [2]:
def SMW_iteration(M, N, b, x, tol = 1.e-5, maxit = 100):
    # M, N : matrices such as A = M - N
    # M : is the first column of the circulant matrix M
    # N : is the sparse matrix N
    # x : guess of the solution
    
    err = 1.0
    iters = 0
    
    while (err > tol and iters < maxit):
        iters += 1
        c = N@x+b
        xnew = spl.solve_circulant(M,c)
        err = npl.norm(xnew-x)
        x = np.copy(xnew)
        
    print('iterations required for convergence:', iters)
    
    return x

# Speed of SMW-iteration in comparison with the size of the system

First we compare the two numerical methods for the simplest case of linear finite element mass matrix. You can choose different dimension $n$ to observe that $n$ increases the SMW-iteration becomes faster than the GMRES method.

In [3]:
n=2000000
M = np.zeros(n)
M[0]=16
M[1]=-5
M[-1]=-5

N = sps.lil_matrix((n,n))
N[0,0]=8; N[n-1,n-1]=8; N[0,n-1]=-5; N[n-1,0]=-5;

xexact = np.ones(n)
x = np.zeros(n)
b = 6.0*np.ones(n)
b[0]=3.0
b[-1]=3.0

%time x = SMW_iteration(M, N, b, x, tol = 1.e-8, maxit = 10000)

print(npl.norm(x-np.ones(n)))

iterations required for convergence: 18
CPU times: user 2.67 s, sys: 446 ms, total: 3.11 s
Wall time: 3.11 s
2.4039471648260297e-09


In [4]:
# This is the GMRES:

from scipy.sparse.linalg import gmres

# Redifine the matrix A for the GMRES routine
A = sps.lil_matrix((n,n))
for i in range(0,n-1):
    A[i,i]=16.0
    A[i,i+1]=-5.0
    A[i+1,i]=-5.0

A[0,0]=8.0
A[n-1,n-1]=8.0

b = 6.0*np.ones(n)
b[0]=3.0
b[-1]=3.0

xexact = np.ones((n,1))
x0 = np.zeros((n,1))

%time (x, info) = gmres(A, b, x0, tol=1.e-8, maxiter=1000, atol=1.e-8)

print(info)
print('The error is:', npl.norm(x-np.ones(n))) 

CPU times: user 14.9 s, sys: 2.68 s, total: 17.6 s
Wall time: 2.61 s
0
The error is: 5.686644391776742e-06


# An extreme random matrix case

This is the setup of a dense system. Try the last two commands only if you take `n` reasonably small.

In [5]:
from numpy.random import rand

n = 1000000

M = 1.0+rand(n)
xexact = np.ones((n,1))
N = sps.lil_matrix((n, n))
N[0,0]=-1.0; N[-1,0]=-1.0; N[0,-1]=-1.0; N[-1,-1]=-1.0;

s = np.sum(M)
b = np.ones((n,1))*s
b[0] -= N[0,0] + N[0,-1]
b[-1] -= N[-1,0] + N[-1,-1]

xexact = np.ones((n,1))
x0 = np.zeros((n,1))

# A = spl.circulant(M)-N
# print('condition number=',npl.cond(A))


In [6]:
# This is our method:

x = SMW_iteration(M, N, b, x0, tol = 1.e-8, maxit = 100)

print('The error is:', npl.norm(x-xexact)) 

iterations required for convergence: 4
The error is: 1.1584307678851722e-09


In the following code we test the python GMRES implementation. I wouldn't try `n=1000000` but you are welcome to do if you like!

In [7]:
# This is the GMRES:

from scipy.sparse.linalg import gmres

n = 100

M = 1.0+rand(n)
xexact = np.ones((n,1))
N = sps.lil_matrix((n, n))
N[0,0]=-1.0; N[-1,0]=-1.0; N[0,-1]=-1.0; N[-1,-1]=-1.0;

s = np.sum(M)
b = np.ones((n,1))*s
b[0] -= N[0,0] + N[0,-1]
b[-1] -= N[-1,0] + N[-1,-1]

xexact = np.ones((n,1))
x0 = np.zeros((n,1))

A = spl.circulant(M)-N
print('condition number=',npl.cond(A))


%time (x, info) = gmres(A, b, x0, tol=1.e-8, maxiter=1000, atol=1.e-8)

print(info)
print('The error is:', npl.norm(x-xexact)) 

condition number= 644.6908732947512
CPU times: user 6.54 s, sys: 1.34 s, total: 7.87 s
Wall time: 1.12 s
1000
The error is: 0.09859878482919404
